# SIDA: Speaker Identification for Archives

In [ ]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio

!mkdir -p /sharedfolder/sida_classifier/training_set/

os.chdir('/sharedfolder/sida_classifier/training_set/')

## Download audio files for training
#### (You may want to comment out the lines below once the download is complete.)
!wget -N http://xtra.arloproject.com/datasets/audio/NPR_Fresh_Air_20_episodes.zip
!unzip NPR_Fresh_Air_20_episodes.zip

In [ ]:
## Download new 1-second labels

csv_url = "https://raw.githubusercontent.com/hipstas/podcast-speaker-labels/master/Fresh_Air/Terry_Gross_labels_randomized.csv"

csv_string = urllib2.urlopen(csv_url)

train_table = []


## Load CSV as list of lists

csv_reader = unicodecsv.reader(csv_string)

for row in csv_reader:
        train_table.append(row)

train_table[:10]+['...']

In [ ]:
## Remove header row (if present)

if 'Media file basename' in train_table[0]:
    train_table = train_table[1:]

In [ ]:
%%capture

## Excerpting WAV clips corresponding to labels
#### (This may take a while.)

training_audio_pathname = "NPR_Fresh_Air_20_episodes"
out_dir = '_classes_' + training_audio_pathname

for row in train_table:
    try:
        basename, start, duration, class_name, labeled_by = row  ## Assigning values in row to variables
        filename = str(basename + '.mp3')
        start = float(start)
        end = float(start) + float(duration)
        wav_out_pathname = str(os.path.join(out_dir, class_name.replace(' ','_')))
        try: 
            subprocess.call(['mkdir', '-p', wav_out_pathname])
        except:
            pass
        attk.subclip(os.path.join(training_audio_pathname, filename), float(start), end, wav_out_pathname) ## <- attk
    except Exception as e: 
        print(row)
        print(e)

In [ ]:
## Defining functions we will use use below

# Extract audio segments to WAV for an audio/video pathname and a list of 2-tuple time values
def extract_vowel_pairs(media_path, vowel_time_ranges):
    snd = AudioFileClip.AudioFileClip(media_path)
    file_duration = attk.duration(media_path)
    for pair in vowel_time_ranges:
        start, end = pair
        start = float(start)
        end = float(end)
        if end-start >= 0.1:  ## Ignore clips shorter than 0.1 second
            basename = media_path.split('/')[-1][:-4]
            out_filename = basename+'__'+str(round(start, 4))+'_'+str(round(end, 4))+'.wav'
            out_pathname = os.path.join('_vowel_clips',out_filename)
            if not os.path.isfile(out_pathname):
                snd.subclip(start, end).write_audiofile(out_pathname)

# Extract vowel segments to WAV for every audio/video file in a given directory
def batch_extract_vowels(media_dir):
    starting_location = os.getcwd()
    os.chdir(media_dir)
    bin_2048_to_sec_constant = 0.046439909297052155
    try: os.mkdir('_vowel_clips')
    except: pass
    filenames=[item for item in os.listdir('./') if item[-4:].lower() in ('.mp3','.wav','.mp4')]
    for filename in filenames:
        try:
            vowel_bools = attk.get_vowel_segments(filename)
            vowel_bin_ranges = attk.labels_to_ranges(vowel_bools, label=True)
            vowel_time_ranges = [(s*bin_2048_to_sec_constant, e*bin_2048_to_sec_constant) for s, e in vowel_bin_ranges]
            extract_vowel_pairs(filename,vowel_time_ranges)
        except Exception as e:
            print("ERROR: " + filename)
            print(e)
    os.chdir(starting_location)

In [ ]:
%%capture
## Extract vowel segments from labeled audio clips
#### (This may take a while.)

os.chdir('/sharedfolder/sida_classifier/training_set/_classes_NPR_Fresh_Air_20_episodes/')

batch_extract_vowels('Terry_Gross')
batch_extract_vowels('Background_Speaker')

In [ ]:
## Extract features (MFCCs, deltas, and delta-deltas) from Terry Gross & UBM vowel clips, then write features to CSVs

os.chdir('/sharedfolder/sida_classifier/training_set/_classes_NPR_Fresh_Air_20_episodes')

dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    try:
        os.chdir('/sharedfolder/sida_classifier/training_set/_classes_NPR_Fresh_Air_20_episodes/' + dir_name + '/_vowel_clips')
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
        filenames = os.listdir('./')
        for filename in filenames:
            csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename)
                    with open(csv_out_path, 'w') as fo:
                        csv_writer = csv.writer(fo)
                        csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        print(e)

In [ ]:
%%capture

## Download and unzip prepared UBM feature set

os.chdir('/sharedfolder/sida_classifier/')

!wget -N https://github.com/hipstas/aapb-ubm/blob/master/UBM_feature_set/AAPB_female_vowel_mfccs_and_deltas.zip
!wget -N https://github.com/hipstas/aapb-ubm/blob/master/UBM_feature_set/AAPB_male_vowel_mfccs_and_deltas.zip
#!wget -N https://raw.githubusercontent.com/hipstas/shaping-humanities-data/master/feature_sets/Terry_Gross_vowel_mfccs_and_deltas.zip
#!wget -N https://raw.githubusercontent.com/hipstas/shaping-humanities-data/master/feature_sets/Fresh_Air_ubm_vowel_mfccs_and_deltas.zip

!unzip AAPB_female_vowel_mfccs_and_deltas.zip
!unzip AAPB_male_vowel_mfccs_and_deltas.zip
#!unzip Terry_Gross_vowel_mfccs_and_deltas.zip
#!unzip Fresh_Air_ubm_vowel_mfccs_and_deltas.zip

In [ ]:
## Load saved features

os.chdir('/sharedfolder/sida_classifier/Terry_Gross_vowel_mfccs_and_deltas')

gross_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            gross_features.append([float(item) for item in row])

print(len(gross_features))


os.chdir('/sharedfolder/sida_classifier/Terry_Gross_vowel_mfccs_and_deltas')

fresh_air_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            fresh_air_ubm_features.append([float(item) for item in row])

print(len(fresh_air_ubm_features))


os.chdir('/sharedfolder/sida_classifier/AAPB_male_vowel_mfccs_and_deltas')

m_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            m_ubm_features.append([float(item) for item in row])

print(len(m_ubm_features))


os.chdir('/sharedfolder/sida_classifier/AAPB_female_vowel_mfccs_and_deltas')

f_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            f_ubm_features.append([float(item) for item in row])

print(len(f_ubm_features))


In [ ]:
## Printing MFCCs and deltas for a single frame

print(random.choice(gross_features))

In [ ]:
## Combining feature sets

speaker_1_mfccs = gross_features
ubm_mfccs = fresh_air_ubm_features + m_ubm_features + f_ubm_features

print(len(speaker_1_mfccs))
print(len(ubm_mfccs))

In [ ]:
## Training Extra Trees Classifier

os.chdir('/sharedfolder/sida_classifier/')

from sklearn.ensemble import ExtraTreesClassifier

X = speaker_1_mfccs[:-len(speaker_1_mfccs)/10] + ubm_mfccs[:-len(ubm_mfccs)/10]
y = [1]*len(speaker_1_mfccs[:-len(speaker_1_mfccs)/10]) + [0]*len(ubm_mfccs[:-len(ubm_mfccs)/10])

X_test = speaker_1_mfccs[-len(speaker_1_mfccs)/10:] + ubm_mfccs[-len(ubm_mfccs)/10:]
y_test = [1]*len(speaker_1_mfccs[-len(speaker_1_mfccs)/10:]) + [0]*len(ubm_mfccs[-len(ubm_mfccs)/10:])

classifier = ExtraTreesClassifier().fit(X, y)

## Saving trained model
joblib.dump(classifier,'gross_vowels_extratrees_2048.pkl')
classifier = joblib.load('gross_vowels_extratrees_2048.pkl')

print(classifier.score(X_test,y_test))

In [ ]:
## Training Simple Multi-Layer Perceptron Model

os.chdir('/sharedfolder/sida_classifier/')

from sklearn.neural_network import MLPClassifier

X = speaker_1_mfccs[:-len(speaker_1_mfccs)/10] + ubm_mfccs[:-len(ubm_mfccs)/10]
y = [1]*len(speaker_1_mfccs[:-len(speaker_1_mfccs)/10]) + [0]*len(ubm_mfccs[:-len(ubm_mfccs)/10])

X_test = speaker_1_mfccs[-len(speaker_1_mfccs)/10:] + ubm_mfccs[-len(ubm_mfccs)/10:]
y_test = [1]*len(speaker_1_mfccs[-len(speaker_1_mfccs)/10:]) + [0]*len(ubm_mfccs[-len(ubm_mfccs)/10:])

#classifier = ExtraTreesClassifier().fit(X, y)
classifier = MLPClassifier().fit(X, y)

## Saving trained model
joblib.dump(classifier,'gross_vowels_mlpc_2048.pkl')
classifier = joblib.load('gross_vowels_mlpc_2048.pkl')

print(classifier.score(X_test,y_test))